In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.cloud import secretmanager

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()
df_v3 = pandas_utils.pandas_from_sql('../sql/v3.sql', con)
con.close()

In [ ]:
df = df_v3.copy()
current_år = df.ÅR.max()
current_måned = df.loc[df.ÅR==df.ÅR.max()].MÅNED.max()
current_år_måned = '-'.join([str(current_år), str(current_måned)])


# Add "ÅR-MÅNED" TO FRONT
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]
df = df.loc[df["ÅR-MÅNED"] != current_år_måned]

df["STØNADSOMRÅDE"] = df.STØNADSOMRÅDE.apply(pesys_utils.map_stonad)

In [ ]:
df["ÅR-MÅNED"].tail()

In [ ]:
df_ap = df.copy().loc[df.STØNADSOMRÅDE=="Alderspensjon"]
df_ap = df_ap.loc[df_ap.BATCH_FLAGG == "ikke opprettet av batch"]

In [ ]:
df_ap = df_ap.merge(df_ap.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
                left_on="ÅR-MÅNED",
                right_on="ÅR-MÅNED",
                suffixes=(""," TOTALT")
        )

In [ ]:
df_ap["ANDEL"] = df_ap.ANTALL.divide(df_ap["ANTALL TOTALT"])

In [ ]:
df_ap.head(10)

# Prep for plott

In [ ]:
nav_colors = (
    "#0067C5",
    "#BA3A26",
    "#06893A",
    "#634689",
    "#FF9100",
    "#66CBEC",
    "#F0C419",
)
pastel = px.colors.qualitative.Pastel

figs = {}

### Automatiseringsgrad

In [ ]:
df_auto = df_ap.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_auto = df_auto[df_auto.AUTOMATISERING == "AUTO"].reset_index(drop=True)
df_auto["ANDEL_PROSENT"] = df_auto["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_auto.tail()

In [ ]:
df_plot = df_auto.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[0], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisk", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[0.2,0.8], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs["autograd"] = fig

### Selvbetjening

In [ ]:
df_selv = df_ap.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "OPPRETTET_AV", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_selv = df_selv[df_selv.OPPRETTET_AV == "bruker"].reset_index(drop=True)
df_selv["ANDEL_PROSENT"] = df_selv["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_selv.tail()

In [ ]:
df_plot = df_selv.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[1], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Selvbetjent", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel selvbetjent", secondary_y=True, range=[0.5,1], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

figs["selvbetjening"] = fig

# Teste andre idéer

In [ ]:
con = pesys_utils.open_pen_connection()

df_v2 = pandas_utils.pandas_from_sql('../sql/v2.sql', con)

con.close()

In [ ]:
df_v2.tail(10)

# Automatisering av selvbetjente saker

In [ ]:
df = df_v2.copy()
current_år = df.ÅR.max()
current_måned = df.loc[df.ÅR==df.ÅR.max()].MÅNED.max()
current_år_måned = '-'.join([str(current_år), str(current_måned)])
df = df.loc[df.ÅR >= current_år - 1]

# Add "ÅR-MÅNED" TO FRONT 
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

# Remove current month
df = df.loc[df["ÅR-MÅNED"] != current_år_måned]

In [ ]:
df_selv = df.loc[df["OPPRETTET_AV"] == "bruker"]
df_selv = df_selv.loc[df_selv["BATCH_FLAGG"] == "ikke opprettet av batch"]

In [ ]:
df_selv_auto = df_selv.groupby(["ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
fig = px.bar(df_selv_auto, "ÅR-MÅNED", "ANTALL", color="AUTOMATISERING", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
fig.show()

figs["selv_auto"] = fig

In [ ]:
df_ikkeselv = df.loc[df["OPPRETTET_AV"] == "saksbehandler"]
df_ikkeselv = df_ikkeselv.loc[df_ikkeselv["BATCH_FLAGG"] == "ikke opprettet av batch"]
df_ikkeselv_auto = df_ikkeselv.groupby(["ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
fig = px.bar(df_ikkeselv_auto, "ÅR-MÅNED", "ANTALL", color="AUTOMATISERING", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
fig.show()

figs["ikkeselv_auto"] = fig

# Utlandstilsnitt

In [ ]:
df_man = df.loc[df["AUTOMATISERING"] == "MAN"]

In [ ]:
df_man_utland = df_man.groupby(["ÅR-MÅNED", "UTLANDSTILSNITT"], as_index=False).ANTALL.sum()

fig = px.bar(df_man_utland, "ÅR-MÅNED", "ANTALL", color="UTLANDSTILSNITT", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["man_utland"] = fig

In [ ]:
df_auto = df.loc[df["AUTOMATISERING"] == "AUTO"]

In [ ]:
df_auto_utland = df_auto.groupby(["ÅR-MÅNED", "UTLANDSTILSNITT"], as_index=False).ANTALL.sum()

fig = px.bar(df_auto_utland, "ÅR-MÅNED", "ANTALL", color="UTLANDSTILSNITT", color_discrete_sequence=nav_colors)
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["auto_utland"] = fig

# Opprettet av batch

In [ ]:
df_man_batch = df_man.groupby(["ÅR-MÅNED", "BATCH_FLAGG"], as_index=False).ANTALL.sum()

fig = px.bar(df_man_batch, "ÅR-MÅNED", "ANTALL", color="BATCH_FLAGG", color_discrete_sequence=nav_colors[2:])
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["man_batch"] = fig
fig.show()

In [ ]:
df_auto_batch = df_auto.groupby(["ÅR-MÅNED", "BATCH_FLAGG"], as_index=False).ANTALL.sum()

fig = px.bar(df_auto_batch, "ÅR-MÅNED", "ANTALL", color="BATCH_FLAGG", color_discrete_sequence=nav_colors[2:])
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)
figs["auto_batch"] = fig
fig.show()

# Sjekk plott

In [ ]:
for _, fig in figs.items():
    fig.show()

# Publiser datastory

In [ ]:
from datastory import DataStory

In [ ]:
ds = DataStory(f"Automatiserings- og selvbetjeningsgrad for alderspensjon")

ds.markdown("**Dette er ikke offisiell statistikk**")
ds.markdown("**Merk at dette er en utdatert og unøyaktig måte å måle automatiseringsgrad.**")
ds.header(f"Automatisering av førstegangsbehandlingssaker for alderspensjon i pesys", level=2)
ds.markdown(f"Denne figuren gjelder kun førstegangsbehandling av alderspensjonssaker. Saker opprettet av batch er eksludert. For noen saker er deler av behandlingen automatisert, men disse regnes som manuelt behandlet.")
ds.plotly(figs["autograd"].to_json())

ds.header(f"Selvbetjening av førstegangsbehandlingssaker for alderspensjon i pesys", level=2)
ds.markdown(f"En sak regnes som selvbetjent dersom den er opprettet av en innbygger (ikke saksbehandler/batch). Denne figuren gjelder kun førstegangsbehandling av alderspensjonssaker. Saker opprettet av batch er eksludert.")
ds.plotly(figs["selvbetjening"].to_json())

# ds.header(f"Blir saker opprettet av bruker automatisert? (Batch eksludert)", level=2)
# ds.markdown(f"Automatisering av saker opprettet av bruker")
# ds.plotly(figs["selv_auto"].to_json())
# ds.markdown(f"Automatisering av saker opprettet av saksbehandler")
# ds.plotly(figs["ikkeselv_auto"].to_json())

# ds.header(f"Hvordan påvirker utlandstilsnitt automatiseringsgrad ved førstegangsbehandling?", level=2)
# ds.markdown(f"Utlandstilsnitt på *manuelt* behandlede saker.")
# ds.plotly(figs["man_utland"].to_json())
# ds.markdown(f"Utlandstilsnitt på *automatisk* behandlede saker.")
# ds.plotly(figs["auto_utland"].to_json())

# ds.header(f"Hvor mange saker opprettet av batch behandles automatisk?")
# ds.markdown(f"*Manuelt* behandlede saker")
# ds.plotly(figs["man_batch"].to_json())
# ds.markdown(f"*Automatisk* behandlede saker")
# ds.plotly(figs["auto_batch"].to_json())


In [ ]:
ds.publish(url="https://nada.intern.nav.no/api") #prod

In [ ]:
#ds.publish(url="https://nada.ekstern.dev.nav.no/api") #dev

# Kladdeområde

In [ ]:
df = df_v4.copy()

In [ ]:
df.head()

In [ ]:
df = df.groupby(["ÅR", "STØNADSOMRÅDE"], as_index=False).ANTALL.sum().sort_values(["ÅR", "ANTALL"], ascending=False).reset_index(drop=True)

In [ ]:
df.head(20)

In [ ]:
px.bar(df, "ÅR", "ANTALL", color="STØNADSOMRÅDE", barmode='group')

In [ ]:
df = df_v4.copy()
current_år = df.ÅR.max()
current_måned = df.loc[df.ÅR==df.ÅR.max()].MÅNED.max()
current_år_måned = '-'.join([str(current_år), str(current_måned)])
df = df.loc[df.ÅR >= current_år-1]

# Add "ÅR-MÅNED" TO FRONT 
df["ÅR-MÅNED"] = df[["ÅR", "MÅNED"]].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
cols = list(df.columns.values)
cols.insert(0, cols.pop())
df = df[cols]

# Remove current month
df = df.loc[df["ÅR-MÅNED"] != current_år_måned].sort_values(["ÅR-MÅNED", "STØNADSOMRÅDE"]).reset_index(drop=True)

In [ ]:
df = df.groupby(["ÅR-MÅNED", "STØNADSOMRÅDE"], as_index=False).ANTALL.sum()

In [ ]:
df.head()

In [ ]:
def get_previous_month(current):
    if current[-1] == 1:
        return "-".join([str(int(current[:4])-1), "12"])
    else:
        return "-".join([current[:4], str(int(current[5:])-1)])
                         
previous_år_måned = get_previous_month(current_år_måned)
df = df.sort_values(["ÅR-MÅNED", "ANTALL"], ascending=False).reset_index(drop=True)

fig = px.area(df, "ÅR-MÅNED", "ANTALL", color="STØNADSOMRÅDE", log_y=True, height=600, )
fig.update_xaxes(type='category')

In [ ]:
list(plot_order)

# Totalt for all kravbehandling

In [ ]:
df = df_v5.copy()

In [ ]:
df.tail()

In [ ]:
current_år = df.ÅR.max()
current_måned = df.loc[df.ÅR==df.ÅR.max()].MÅNED.max()
current_år_måned = '-'.join([str(current_år), str(current_måned)])

df = df.loc[df["ÅR-MÅNED"] != current_år_måned]
df = df[df.KRAVTYPE != "Revurdering"]

In [ ]:
df = df.merge(df.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
                left_on="ÅR-MÅNED",
                right_on="ÅR-MÅNED",
                suffixes=(""," TOTALT")
        )

df["ANDEL"] = df.ANTALL.divide(df["ANTALL TOTALT"])

In [ ]:
df_auto_tot = df.groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING", "ANTALL TOTALT"], as_index=False)[["ANTALL", "ANDEL"]].sum()
df_auto_tot = df_auto_tot[df_auto_tot.AUTOMATISERING == "AUTO"].reset_index(drop=True)
df_auto_tot["ANDEL_PROSENT"] = df_auto_tot["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

In [ ]:
df_auto_tot.head()

In [ ]:
df_plot = df_auto_tot.copy()

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_plot["ÅR-MÅNED"], y=df_plot["ANTALL TOTALT"], marker_color=pastel[0], name="Antall saker"), secondary_y=False)

fig.add_trace(go.Scatter(x=df_plot["ÅR-MÅNED"], y=df_plot["ANDEL"], text=df_plot["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisert", textposition='top center'), secondary_y=True)
fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[-0.1,1.1], tickformat='.0%')
fig.update_yaxes(title_text="Antall saker", secondary_y=False)
fig.layout.yaxis2.showgrid = False
fig.update_layout(
    autosize=False,
    width=1300,
    height=500,)

# Automatiseringsgrad per stønad og kravtype

In [ ]:
df_all = df_v5.copy()
df_all = df_all.loc[df_all["ÅR-MÅNED"] != current_år_måned]

# Alle interessante måneder:
df_months = pd.DataFrame(list(np.unique(df_all["ÅR-MÅNED"])), columns=["ÅR-MÅNED"])
df_months[['ÅR', 'MÅNED']] = df_months['ÅR-MÅNED'].str.split('-', 1, expand=True).astype(int)
df_months = df_months.sort_values(["ÅR", "MÅNED"]).reset_index(drop=True)

In [ ]:
df_all.head()

## Alle stønadsområder
- AFP
- AFP Privat
- Alderspensjon
- Barnepensjon
- Familiepleierytelse
- Gammel yrkesskade
- Generell
- Gjenlevendeytelse
- Krigspensjon
- Omsorgsopptjening
- Uføretrygd

### Mulige kravtyper
- AFP etteroppgjør
- Anke
- Eksport
- Endring uttaksgrad
- Erstatning
- Ettergivelse av gjeld
- Førstegangsbehandling
- Førstegangsbehandling Norge/utland
- Førstegangsbehandling bosatt utland'
- Førstegangsbehandling kun utland
- Godskriving omsorgsopptjening
- Hjelpeberegning ved overgang til uføretrygd
- Inntektsendring
- Klage
- Mellombehandling
- Omgjøring av tilbakekreving
- Overføring omsorgsopptjening
- ReguleringRevurdering
- Saksomkostninger
- Sluttbehandling Norge/utland
- Sluttbehandling kun utland
- Søknad om reduksjon av uføregrad
- Søknad om ung ufør
- Søknad om yrkesskade
- Søknad om økning av uføregrad
- Tilbakekreving
- Uføretrygd etteroppgjør
- Uføretrygd vurdering av etteroppgjør
- Utsendelse til avtaleland

In [ ]:
#stonader = np.unique(df_all.STØNADSOMRÅDE)
stonader = ["Alderspensjon"]
#kravtyper = "All"
kravtyper = ["Eksport"]


for stonad in stonader:
    df_stonad = df_all[df_all.STØNADSOMRÅDE == stonad]
    if kravtyper == "All":
        loop = np.unique(df_stonad.KRAVTYPE)
    else:
        loop = kravtyper
    for kravtype in loop:
        df_kravtype = df_stonad.loc[df_stonad.KRAVTYPE == kravtype].groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
        df_plot = df_kravtype.merge(df_kravtype.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
                left_on="ÅR-MÅNED",
                right_on="ÅR-MÅNED",
                suffixes=(""," TOTALT")
        )
        df_plot_auto = df_plot.loc[df_plot.AUTOMATISERING == "AUTO"].sort_values(["ÅR", "MÅNED"]).reset_index(drop=True)
        df_plot_man = df_plot.loc[df_plot.AUTOMATISERING == "MAN"].sort_values(["ÅR", "MÅNED"]).reset_index(drop=True)

        df_plot_auto["ANDEL"] = df_plot_auto.ANTALL.divide(df_plot_auto["ANTALL TOTALT"])
        df_plot_auto["ANDEL_PROSENT"] = df_plot_auto["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'
        
        #df_plot_auto = df_plot_auto.set_index("ÅR-MÅNED").reindex(df_months["ÅR-MÅNED"], fill_value=0).reset_index()
        
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        fig.add_trace(go.Bar(x=df_plot_auto["ÅR-MÅNED"], y=df_plot_auto["ANTALL"], marker_color=pastel[4], name="Automatisk"), secondary_y=False)
        fig.add_trace(go.Bar(x=df_plot_man["ÅR-MÅNED"], y=df_plot_man["ANTALL"], marker_color=pastel[2], name="Manuelt"), secondary_y=False)

        fig.add_trace(go.Scatter(x=df_plot_auto["ÅR-MÅNED"], y=df_plot_auto["ANDEL"], text=df_plot_auto["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisert", textposition='top center'), secondary_y=True)
        fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[-0.1,1.1], tickformat='.0%')
        fig.update_yaxes(title_text="Antall saker", secondary_y=False)
        fig.layout.yaxis2.showgrid = False
        fig.update_layout(
            autosize=False,
            width=1300,
            height=500,
            barmode="stack")
        print(stonad, "-", kravtype)
        fig.show()

In [ ]:
df_plot_man.head()

In [ ]:
df_rank = df_v5.copy().groupby(["STØNADSOMRÅDE", "KRAVTYPE"], as_index=False).ANTALL.sum().sort_values("ANTALL", ascending=False).reset_index(drop=True)

In [ ]:
df_rank.head(10)

In [ ]:
n_ytelser = 20

for i in range(n_ytelser):
    stonad = df_rank.iloc[i,0]
    kravtype = df_rank.iloc[i,1]

    df_stonad = df_all[df_all.STØNADSOMRÅDE == stonad]
    df_kravtype = df_stonad.loc[df_stonad.KRAVTYPE == kravtype].groupby(["ÅR", "MÅNED", "ÅR-MÅNED", "AUTOMATISERING"], as_index=False).ANTALL.sum()
    
    df_plot = df_kravtype.merge(df_kravtype.groupby("ÅR-MÅNED", as_index=False).ANTALL.sum(),
            left_on="ÅR-MÅNED",
            right_on="ÅR-MÅNED",
            suffixes=(""," TOTALT")
    )
    df_plot_auto = df_plot.loc[df_plot.AUTOMATISERING == "AUTO"].sort_values(["ÅR", "MÅNED"]).reset_index(drop=True)
    df_plot_man = df_plot.loc[df_plot.AUTOMATISERING == "MAN"].sort_values(["ÅR", "MÅNED"]).reset_index(drop=True)

    df_plot_auto["ANDEL"] = df_plot_auto.ANTALL.divide(df_plot_auto["ANTALL TOTALT"])
    df_plot_auto["ANDEL_PROSENT"] = df_plot_auto["ANDEL"].apply(lambda x: round(x*100, 0)).astype(int).astype(str) + '%'

    df_plot_auto = df_plot_auto.set_index("ÅR-MÅNED").reindex(df_months["ÅR-MÅNED"], fill_value=0).reset_index()


    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=df_plot_auto["ÅR-MÅNED"], y=df_plot_auto["ANTALL"], marker_color=pastel[4], name="Automatisk"), secondary_y=False)
    fig.add_trace(go.Bar(x=df_plot_man["ÅR-MÅNED"], y=df_plot_man["ANTALL"], marker_color=pastel[2], name="Manuelt"), secondary_y=False)

    fig.add_trace(go.Scatter(x=df_plot_auto["ÅR-MÅNED"], y=df_plot_auto["ANDEL"], text=df_plot_auto["ANDEL_PROSENT"], mode='lines+markers+text', marker_color="black", name="Automatisert", textposition='top center'), secondary_y=True)
    fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[-0.1,1.1], tickformat='.0%')
    fig.update_yaxes(title_text="Antall saker", secondary_y=False)
    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=False,
        width=1300,
        height=500,
        barmode="stack")
    print(stonad, "-", kravtype)
    fig.show()